### Polynomial Linear Regression

In [190]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import PolynomialFeatures
import seaborn as sns

In [191]:
euro_c_df_clean = pd.read_csv('euro_c_df_clean.csv')

# remove observations with frp greater than ______
# outlier_indices = euro_c_df_clean[(euro_c_df_clean['frp'] 208)].index
# euro_c_df_clean = euro_c_df_clean.drop(index=outlier_indices)


euro_c_df_clean['frp'].max()

134.9

In [192]:
# date -> days since
date = euro_c_df_clean['acq_date']
x_datetime = pd.to_datetime(date)
reference_date = pd.Timestamp(date.min())       # earliest date, '2020-03-01'
x_timedelta = x_datetime - reference_date
# number of days between data collection and reference date
x_days = x_timedelta.dt.days

# scale data
x_days = (x_days - x_days.mean())/x_days.std()
x_days = x_days.to_numpy()
frp=euro_c_df_clean['frp'].to_numpy()
frp = (frp - frp.mean())/frp.std()
lat = euro_c_df_clean['latitude'].to_numpy()
long = euro_c_df_clean['longitude'].to_numpy()
lat = ((lat - lat.mean())/lat.std())
long = ((long - long.mean())/long.std())

y = frp      # y: frp
min(y)


-0.6264380367467925

In [202]:
# lat, long, acq_date
X2 = np.hstack([
    np.ones((x_days.shape[0], 1)), 
    lat.reshape(-1, 1), 
    lat.reshape(-1, 1)**2,
    lat.reshape(-1, 1)**3,
    lat.reshape(-1, 1)**4,
    lat.reshape(-1, 1)**5,
    long.reshape(-1, 1), 
    long.reshape(-1, 1)**2,
    long.reshape(-1, 1)**3,
    long.reshape(-1, 1)**4, 
    long.reshape(-1, 1)**5,
    (lat.reshape(-1, 1)*long.reshape(-1, 1)),
    (lat.reshape(-1, 1)*long.reshape(-1, 1))**2,
    (lat.reshape(-1, 1)*long.reshape(-1, 1))**3,
    (lat.reshape(-1, 1)*long.reshape(-1, 1))**4,
    (lat.reshape(-1, 1)*long.reshape(-1, 1))**5,
    x_days.reshape(-1, 1),
        x_days.reshape(-1, 1)**2,
            x_days.reshape(-1, 1)*3,
                x_days.reshape(-1, 1)**4,
                    x_days.reshape(-1, 1)**5
    ])
X2

array([[ 1.00000000e+00,  1.50561775e-06, -2.96681612e-08,
        -4.46689100e-14, -3.96115171e+00],
       [ 1.00000000e+00,  1.00918494e-05, -4.36714032e-10,
        -4.40725222e-15, -3.96115171e+00],
       [ 1.00000000e+00,  1.01171530e-05, -4.39001606e-10,
        -4.44144639e-15, -3.96115171e+00],
       ...,
       [ 1.00000000e+00,  2.14932386e+01,  2.87406227e-01,
         6.17729059e+00,  2.71717412e+00],
       [ 1.00000000e+00,  1.86851907e+01,  1.34822442e-03,
         2.51918304e-02,  2.71717412e+00],
       [ 1.00000000e+00,  2.14604700e+02,  4.14439179e+00,
         8.89405959e+02,  6.95842977e+00]])

In [203]:
# use np math to find m
m = np.matmul(np.linalg.inv(np.matmul(X2.T, X2)), np.matmul(X2.T, y))
print('m: ', m)

# calculate predicted y's, residuals, squared residuals, mse, and r^2
ypreds = np.matmul(X2, m)
resids = y - ypreds
new_r = []
for r in resids:
    new_r.append(r**2)
mse = sum(new_r) / len(new_r)
print('mse: ', mse)
r2 = 1 - (mse/y.var())
print('r^2: ', r2)


m:  [ 9.36380895e-03 -1.17771292e-03 -1.30930831e-05 -1.21491250e-05
 -2.88561077e-02]
mse:  0.9940879424075594
r^2:  0.005912057592440845


In [195]:
# # PLOTS - lat, long, acq_date
# print(f'Best-fit line: y = {round(m[0], 3)} + {round(m[1], 3)}x + {round(m[2], 3)}x + {round(m[3], 3)}x')
# n = ['lat', 'long', 'acq_date']
# for i in range(len(X2)):
#     x_plot = X2[:,i]
#     plt.scatter(x=(x_plot), y=y, alpha=0.7, edgecolor='k')
#     # plt.plot(x_plot, m[0] + m[1]*x_plot + m[2]*x_plot + m[3]*x_plot)
#     # plt.title(f'{i} vs. frp', fontsize=14)
#     # plt.xlabel(f'{i}', fontsize=12)
#     # plt.ylabel('frp', fontsize=12)
#     plt.grid(alpha=0.3)
#     plt.show()

In [196]:
# # index vs. residuals
# plt.scatter(range(len(X2[:,1])), resids, alpha=0.5)
# plt.xlabel('Index')
# plt.ylabel('Residuals')
# plt.title('Index vs. Residuals')
# plt.show()

In [197]:
# # histogram of residuals
# sns.histplot(resids, kde=False)
# plt.xlabel('Residuals')
# plt.title('Histogram of residuals')
# plt.show()